In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/topic-modeling-for-research-articles/train.csv
/kaggle/input/topic-modeling-for-research-articles/test.csv


In [2]:
import pandas as pd 
import numpy as np


In [3]:
df=pd.read_csv('/kaggle/input/topic-modeling-for-research-articles/train.csv')


In [4]:
df.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [5]:
import string
exclude=string.punctuation

In [6]:
exclude

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [7]:
import spacy

nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])

/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():


In [8]:
def functions (df):
    df=df.drop('ID',axis=1)
    # df['TITLE']=df['TITLE'].apply(function2)
    df['text']=df['TITLE']+' '+df['TITLE']+' '+df['ABSTRACT']
    def function2(text):
        text=text.strip()
        text=text.lower()
        for i in exclude:
            text=text.replace(i,'')
        
        return text
    df['text']=df['text'].apply(function2)
    df=df.drop(['TITLE','ABSTRACT'],axis=1)
    
    
    return df

In [9]:
df=functions(df)

In [10]:
df.columns

Index(['Computer Science', 'Physics', 'Mathematics', 'Statistics',
       'Quantitative Biology', 'Quantitative Finance', 'text'],
      dtype='object')

In [11]:
numeric_df = df.select_dtypes(include=["number"])


In [12]:
ne=numeric_df.columns

In [13]:
def subjects_to_text(row):
    subjects = []
    for col in ne:
        if row[col] == 1:
            subjects.append("subject_" + col.lower().replace(" ", "_"))
    return " ".join(subjects)


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20972 entries, 0 to 20971
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Computer Science      20972 non-null  int64 
 1   Physics               20972 non-null  int64 
 2   Mathematics           20972 non-null  int64 
 3   Statistics            20972 non-null  int64 
 4   Quantitative Biology  20972 non-null  int64 
 5   Quantitative Finance  20972 non-null  int64 
 6   text                  20972 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.1+ MB


In [15]:
df.head()

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,text
0,1,0,0,0,0,0,reconstructing subjectspecific effect maps rec...
1,1,0,0,0,0,0,rotation invariance neural network rotation in...
2,0,0,1,0,0,0,spherical polyharmonics and poisson kernels fo...
3,0,0,1,0,0,0,a finite element approximation for the stochas...
4,1,0,0,1,0,0,comparative study of discrete wavelet transfor...


In [16]:
df["subject_text"] = df.apply(subjects_to_text, axis=1)

df["text"] = df["text"] + " " + df["subject_text"]


In [17]:
df['text'][0]

'reconstructing subjectspecific effect maps reconstructing subjectspecific effect maps   predictive models allow subjectspecific inference when analyzing disease\nrelated alterations in neuroimaging data given a subjects data inference can\nbe made at two levels global ie identifiying condition presence for the\nsubject and local ie detecting condition effect on each individual\nmeasurement extracted from the subjects data while global inference is widely\nused local inference which can be used to form subjectspecific effect maps\nis rarely used because existing models often yield noisy detections composed of\ndispersed isolated islands in this article we propose a reconstruction\nmethod named rsm to improve subjectspecific detections of predictive\nmodeling approaches and in particular binary classifiers rsm specifically\naims to reduce noise due to sampling error associated with using a finite\nsample of examples to train classifiers the proposed method is a wrappertype\nalgorithm th

In [18]:
tf=df['text']

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
vectorizer = CountVectorizer(
    lowercase=True,
    stop_words="english",
    min_df=40,
    max_df=0.9,
    ngram_range=(1, 4),
    token_pattern=r'(?u)\b[a-zA-Z]{2,}\b'
)
x=vectorizer.fit_transform(df['text'])


In [21]:
x=vectorizer.fit_transform(df['text'])

In [22]:
x

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 1208990 stored elements and shape (20972, 5596)>

In [23]:
len(vectorizer.get_feature_names_out())


5596

In [24]:
vectorizer.get_feature_names_out()[:20]


array(['ab', 'ab initio', 'abelian', 'abilities', 'ability', 'able',
       'absence', 'absent', 'absolute', 'absorption', 'abstract',
       'abstraction', 'abundance', 'abundances', 'abundant', 'academic',
       'accelerate', 'accelerated', 'accelerating', 'acceleration'],
      dtype=object)

In [25]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(
    n_components=6,        # start with number of subjects
    doc_topic_prior=0.1,
    topic_word_prior=0.01,
    learning_method="batch",
    max_iter=20,
    random_state=42,
    n_jobs=-1
)

lda.fit(x)


LatentDirichletAllocation(doc_topic_prior=0.1, max_iter=20, n_components=6,
                          n_jobs=-1, random_state=42, topic_word_prior=0.01)

In [26]:
feature_names = vectorizer.get_feature_names_out()

for idx, topic in enumerate(lda.components_):
    top_words = [feature_names[i] for i in topic.argsort()[-12:]]
    print(f"\nTopic {idx}: {top_words}")



Topic 0: ['star', 'galaxy', 'results', 'emission', 'high', 'dark', 'stars', 'observations', 'galaxies', 'data', 'using', 'mass']

Topic 1: ['systems', 'based', 'time', 'paper', 'using', 'models', 'information', 'analysis', 'networks', 'network', 'model', 'data']

Topic 2: ['using', 'proposed', 'control', 'model', 'approach', 'learning', 'methods', 'optimization', 'algorithms', 'problem', 'method', 'algorithm']

Topic 3: ['classification', 'models', 'neural networks', 'using', 'model', 'training', 'data', 'network', 'networks', 'deep', 'neural', 'learning']

Topic 4: ['study', 'graph', 'function', 'equations', 'theory', 'results', 'finite', 'space', 'prove', 'functions', 'paper', 'problem']

Topic 5: ['transition', 'spin', 'systems', 'states', 'state', 'field', 'dynamics', 'energy', 'magnetic', 'phase', 'model', 'quantum']


In [27]:
test=pd.read_csv('/kaggle/input/topic-modeling-for-research-articles/test.csv')

In [28]:
test=functions(test)


In [29]:
x=vectorizer.transform(df['text'])


In [30]:
doc_topic_test = lda.transform(x)


In [31]:
test_perplexity = lda.perplexity(x)
print("Test Perplexity:", test_perplexity)


Test Perplexity: 1907.4956548467171
